In [7]:
#Importing the Relevant Modules
import pandas as pd
import censusdata
from census import Census
from us import states
import numpy as np

In [8]:
#Include invariants, i.e, variables of interest, acs, etc.
acs_year = 2018
#set up API key
key = "40a3bb886bca60b069f4ec70dbcd69176acf4faf"


In [9]:
#Searching for Possible Variables of Interest


#possible fields: name (code), label (descriptor), concept (education, etc)
#censusdata.search(src='acs5', year=acs_year, field='concept', criterion='race')

#create list of desired vars
race_vars = ["B02001_002E", "B02001_003E", "B02001_004E", "B02001_005E", "B02001_006E", "B02001_007E", "B02001_008E"]
education_vars = ["B15003_002E", "B15003_003E", "B15003_004E", "B15003_005E", "B15003_006E", "B15003_007E", "B15003_008E", 
                  "B15003_009E", "B15003_010E", "B15003_011E",
"B15003_012E", "B15003_013E", "B15003_014E", "B15003_015E", "B15003_016E", "B15003_017E", "B15003_018E", "B15003_019E", "B15003_020E", "B15003_021E",
"B15003_022E", "B15003_023E", "B15003_024E", "B15003_025E"]
family_hh_income = ["B19101_002E", "B19101_003E", "B19101_004E", "B19101_005E", "B19101_006E", "B19101_007E", "B19101_008E", "B19101_009E", "B19101_010E", "B19101_011E",
"B19101_012E", "B19101_013E", "B19101_014E", "B19101_015E", "B19101_016E", "B19101_017E"]
latino_population = ["B03002_002E", "B03002_012E"]
#total_population = ["B00001_001E", "B00002_001E"]
other_cols = ['NAME']
census_var_list = race_vars + education_vars + family_hh_income + latino_population
census_var_download_list = other_cols + census_var_list
census_var_download_list = tuple(census_var_download_list)


#set up geography of interest

#look up the possible geographies
#censusdata.geographies(censusdata.censusgeo([('state', '*')]), 'acs5', acs_year)
#censusdata.geographies(censusdata.censusgeo([('state', '06'), ('county', '*')]), 'acs5', acs_year)

#finally declare the actual geography
#geo = censusdata.censusgeo(geo=[('state', '06'), ('county', '037')]

In [10]:
#download the census data
c = Census(key=key, year = acs_year)

#at the zip code level
#census_data_zcta = c.acs5.get(fields = census_var_download_list, geo = {'for': 'zip code tabulation area:*'})
                                                               
#at the tract level
#census_data_tract = c.acs5.get(fields = census_var_download_list, geo = {'for': 'tract:*' , 
#                                                               'in': 'state:{} county:{}'.format('06', '037')})


#at the block group level
census_data_block_group = c.acs5.get(fields = census_var_download_list, geo = {'for': 'block group:*' , 
                                                                       'in': 'state:{} county:{}'.format('06', '037')})


In [11]:
#format the census data
#census_data_formatted_zcta = pd.DataFrame(census_data_zcta)
#census_data_formatted_zcta['NAME'] = census_data_formatted_zcta['NAME'].str.replace('ZCTA5 ','')
#census_data_formatted_zcta = census_data_formatted_zcta.melt(id_vars=['NAME','zip code tabulation area'],
#       value_vars=census_var_list)

#census_data_formatted_tract = pd.DataFrame(census_data_tract)
#census_data_formatted_tract['NAME'] = census_data_formatted_tract['NAME'].str.replace('ZCTA5 ','')
#census_data_formatted_tract = census_data_formatted_tract.melt(id_vars=['NAME','zip code tabulation area'],
#       value_vars=census_var_list)

census_data_formatted_block_group = pd.DataFrame(census_data_block_group)
census_data_formatted_block_group = census_data_formatted_block_group.melt(id_vars=['NAME','state', 'county', 'tract', 'block group'],
       value_vars=census_var_list)

In [12]:
#remaining aggregations
#change variable to actual human readible variable
#need to do the following:
#If variable has B02001, then its a Race variable
#If variable has B15003, then its an Education
#Else, it's an Income variable
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B02001"),'Variable Type'] = 'Race'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003"),'Variable Type'] = 'Education'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101"),'Variable Type'] = 'Family HH Income'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B03002"),'Variable Type'] = 'Hispanic or Latino'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B00001"),'Variable Type'] = 'Population'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B00002"),'Variable Type'] = 'Housing Units'

#Now, Define Variable Sub Type
#First, for Race
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B02001_002"),'Variable Subtype'] = 'White'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B02001_003"),'Variable Subtype'] = 'Black'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B02001_004"),'Variable Subtype'] = 'American Indian or Alaskan'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B02001_005"),'Variable Subtype'] = 'Asian'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B02001_006"),'Variable Subtype'] = 'Native Hawaiian and Other Pacific Islander'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B02001_007"),'Variable Subtype'] = 'Some other race'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B02001_008"),'Variable Subtype'] = 'Two or more races'
#remove the 009 and 010

#Then, for Education
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_002"),'Variable Subtype'] = 'No Schooling'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_003"),'Variable Subtype'] = 'Pre-K'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_004"),'Variable Subtype'] = 'K-8'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_005"),'Variable Subtype'] = 'K-8'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_006"),'Variable Subtype'] = 'K-8'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_007"),'Variable Subtype'] = 'K-8'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_008"),'Variable Subtype'] = 'K-8'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_009"),'Variable Subtype'] = 'K-8'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_010"),'Variable Subtype'] = 'K-8'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_011"),'Variable Subtype'] = 'K-8'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_012"),'Variable Subtype'] = 'K-8'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_013"),'Variable Subtype'] = 'Some High School'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_014"),'Variable Subtype'] = 'Some High School'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_015"),'Variable Subtype'] = 'Some High School'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_016"),'Variable Subtype'] = 'Some High School'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_017"),'Variable Subtype'] = 'HS Diploma / Equivalent'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_018"),'Variable Subtype'] = 'HS Diploma / Equivalent'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_019"),'Variable Subtype'] = 'Some college'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_020"),'Variable Subtype'] = 'Some college'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_021"),'Variable Subtype'] = 'Associates degree'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_022"),'Variable Subtype'] = 'Bachelors degree'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_023"),'Variable Subtype'] = 'Masters degree'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_024"),'Variable Subtype'] = 'Professional School Degree'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B15003_025"),'Variable Subtype'] = 'Doctorate'

#Finally, for Income
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_002"),'Variable Subtype'] = 'Very Low Income'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_003"),'Variable Subtype'] = 'Very Low Income'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_004"),'Variable Subtype'] = 'Very Low Income'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_005"),'Variable Subtype'] = 'Very Low Income'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_006"),'Variable Subtype'] = 'Very Low Income'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_007"),'Variable Subtype'] = 'Very Low Income'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_008"),'Variable Subtype'] = 'Low Income'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_009"),'Variable Subtype'] = 'Low Income'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_010"),'Variable Subtype'] = 'Low Income'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_011"),'Variable Subtype'] = 'Low Income'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_012"),'Variable Subtype'] = 'Middle Class'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_013"),'Variable Subtype'] = 'Middle Class'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_014"),'Variable Subtype'] = 'Middle Class'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_015"),'Variable Subtype'] = 'Middle Class'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_016"),'Variable Subtype'] = 'Middle Class'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B19101_017"),'Variable Subtype'] = 'High Income'

#jk, need latino
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B03002_002"),'Variable Subtype'] = 'Not Hispanic or Latino'
census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B03002_012"),'Variable Subtype'] = 'Hispanic or Latino'

#jk again, include total population explicitly
#census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B00001_001"),'Variable Subtype'] = 'Population'
#census_data_formatted_block_group.loc[census_data_formatted_block_group['variable'].str.contains("B00002_001"),'Variable Subtype'] = 'Housing Units'

#census_data_formatted_block_group['Variable Subtype'] 
census_data_formatted_block_group.head()

,NAME,state,county,tract,block group,variable,value,Variable Type,Variable Subtype
0,"Block Group 1, Census Tract 1011.10, Los Angel...",06,037,101110,1,B02001_002E,1550.0,Race,White
1,"Block Group 2, Census Tract 1011.10, Los Angel...",06,037,101110,2,B02001_002E,985.0,Race,White
2,"Block Group 3, Census Tract 1011.10, Los Angel...",06,037,101110,3,B02001_002E,779.0,Race,White
3,"Block Group 1, Census Tract 1011.22, Los Angel...",06,037,101122,1,B02001_002E,1812.0,Race,White
4,"Block Group 2, Census Tract 1011.22, Los Angel...",06,037,101122,2,B02001_002E,1025.0,Race,White


In [13]:
census_data_formatted_block_group.to_csv("census_data_block_group.csv")